# Pilot 2 -- Compute the baseline decodability of various input stimuli combinations
### Ex: Color stims ([red, red] v. [red, blue] v. [blue, red] v. [blue, blue])

## Use SVM classifications to input stimuli (both modalities: visual and auditory)
## Using Ciric-style postprocessing

## Takuya Ito
#### 12/06/2018

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns
import h5py
os.sys.path.append('glmScripts/')
import taskGLMPipeline as tgp

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [3]:
def loadInputActivity(subj,inputtype):
    x = tgp.loadTaskTiming(subj,'ALL')
    stimIndex = np.asarray(x['stimIndex'])
    ind = np.where(stimIndex==inputtype)[0]
    
    datadir = basedir + 'data/postProcessing/hcpPostProcCiric/'
    h5f = h5py.File(datadir + subj + '_glmOutput_data.h5','r')
    data = h5f['taskRegression/ALL_24pXaCompCorXVolterra_taskReg_betas_canonical'][:].copy()
    data = data[:,ind].copy()
    h5f.close()
    return data


# Univariate analyses to identify regions sensitive to stimulus activity

In [4]:
stimulusActivations = np.zeros((len(glasser2),len(subjNums)))
scount = 0
for subj in subjNums:
    stimulusActivations[:,scount] = np.mean(loadInputActivity(subj,'colorStim'),axis=1)
    scount += 1

In [5]:
roiActivity = np.zeros((nParcels,))
for roi in range(nParcels):
    roi_ind = np.where(glasser2==roi+1)[0]
    roi_activity = np.mean(stimulusActivations[roi_ind,:],axis=0)
    t, p = stats.ttest_1samp(roi_activity,0)
    roiActivity[roi] = p


h0, q = mc.fdrcorrection0(roiActivity)
roiActivity[:] = h0
print 'Number of significant activations during stimulus presentation:', np.sum(h0)
activeROIs = np.where(h0)[0] + 1

activeROIs = np.arange(1,361)
    
        

Number of significant activations during stimulus presentation: 219


# 0.1 Define functions for input stimuli decodings

In [11]:
from scipy.spatial.distance import cdist

def inputStimuliDecoding(data, inputtype, atlasROIs=np.arange(1,nParcels+1), ncvs=100, nproc=5):
    """
    Run an across-subject classification
    Decode input stimuli (4-way decoding) for each sensory rule type
    Limit to ROIs within either vis or aud network
    """
        
    nstim = 4 # Number of stimuli per stimulus-type
    
    nSubjs = data.shape[2]
    if inputtype in ['colorStim','oriStim']:
        # Visual networks
        rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
        rois = rois + 1
        rois = np.intersect1d(rois,atlasROIs)
    elif inputtype in ['constantStim','pitchStim']:
        rois = np.where(networkdef==networkmappings['aud'])[0]
        rois = rois + 1
        rois = np.intersect1d(rois,atlasROIs)

    netStats = np.zeros((len(rois),nstim*nSubjs))


    nsamples = nSubjs * nstim

    # Label array for supervised learning
    labels = np.tile(range(nstim),nSubjs)
    subjarray = np.repeat(range(nSubjs),nstim)

    # Run SVM classifications on network-level activation patterns across subjects
    roicount = 0
    for roi in rois:
        print 'Computing SVM classification for input stimuli:', inputtype, 'for ROI', roicount, '/', len(rois)
        roi_ind = np.where(glasser2==roi)[0]
        nfeatures = len(roi_ind)
        roi_ind.shape = (len(roi_ind),1)       

        svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
        samplecount = 0
        scount = 0
        for subj in range(len(subjNums)):
            roidata = np.squeeze(data[roi_ind,:,scount])
            svm_mat[samplecount:(samplecount+nstim),:] = roidata.T

            scount += 1
            samplecount += nstim

            # Spatially demean matrix across features
            samplemean = np.mean(svm_mat,axis=1)
            samplemean.shape = (len(samplemean),1)
            svm_mat = svm_mat - samplemean

        scores = randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=nproc)
        netStats[roicount,:] = scores
        roicount += 1


    return netStats

def randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=5):
    """
    Runs cross validation for an across-subject SVM analysis
    """
    
    ntasks = len(np.unique(labels))
    nsamples = svm_mat.shape[0]
    nsubjs = nsamples/ntasks

    subjects = np.unique(subjarray)
    indices = np.arange(nsamples)
    
    numsubjs_perfold = 1
    if nsubjs%numsubjs_perfold!=0: 
        raise Exception("Error: Folds don't match number of subjects")
        
    nfolds = nsubjs/numsubjs_perfold
    subj_array_folds = subjarray.copy()
    
    inputs = [] 
    
    for fold in range(nfolds):
        test_subjs = np.random.choice(subj_array_folds,numsubjs_perfold,replace=False)
        train_subjs_all = np.delete(subjects,test_subjs)
        for cv in range(ncvs):
            # Randomly sample half of train set subjects for each cv (CV bootstrapping)
            train_subjs = np.random.choice(train_subjs_all,
                                         int(np.floor(len(train_subjs_all)*(4.0))),
                                         replace=True)

            train_ind = []
            for subj in train_subjs:
                train_ind.extend(np.where(subjarray==subj)[0])

            test_ind = []
            for subj in test_subjs:
                test_ind.extend(np.where(subjarray==subj)[0])
            
            train_ind = np.asarray(train_ind)
            test_ind = np.asarray(test_ind)

            trainset = svm_mat[train_ind,:]
            testset = svm_mat[test_ind,:]

            # Normalize trainset and testset
            mean = np.mean(svm_mat[train_ind,:],axis=0)
            mean.shape = (1,len(mean))
            std = np.std(svm_mat[train_ind,:],axis=0)
            std.shape = (1,len(std))

            trainset = np.divide((trainset - mean),std)
            testset = np.divide((testset - mean),std)
            
            ## Feature selection and downsampling
            trainlabels = labels[train_ind]
            testlabels = labels[test_ind]
            unique_labels = np.unique(labels)
            feat1_labs = np.where(trainlabels==unique_labels[0])[0]
            feat2_labs = np.where(trainlabels==unique_labels[1])[0]
            # Perform t-test
            t, p = stats.ttest_rel(trainset[feat1_labs,:],trainset[feat2_labs,:],axis=0)
            h0, qs = mc.fdrcorrection0(p)
#             h0 = p<0.1
            # Construct feature masks
            feat1_mask = np.multiply(t>0,h0).astype(bool)
            feat2_mask = np.multiply(t<0,h0).astype(bool)
#             feat1_mask = t>0
#             feat2_mask = t<0
            # Downsample training set into original vertices into 2 ROI signals
            trainset_downsampled = np.zeros((trainset.shape[0],2))
            trainset_downsampled[:,0] = np.nanmean(trainset[:,feat1_mask],axis=1)
            trainset_downsampled[:,1] = np.nanmean(trainset[:,feat2_mask],axis=1)
            trainset_downsampled = trainset[:,h0]
            # Downsample test set into original vertices
            testset_downsampled = np.zeros((testset.shape[0],2))
            testset_downsampled[:,0] = np.nanmean(testset[:,feat1_mask],axis=1)
            testset_downsampled[:,1] = np.nanmean(testset[:,feat2_mask],axis=1)
            testset_downsampled = testset[:,h0]
            
            if np.sum(feat1_mask)==0 or np.sum(feat2_mask==0):
                inputs.append((trainset,testset,trainlabels,testlabels))
            else:
                inputs.append((trainset_downsampled,testset_downsampled,trainlabels,testlabels))
                
#             inputs.append((trainset,testset,trainlabels,testlabels))
            
        
        subj_array_folds = np.delete(subj_array_folds,test_subjs)
        
    pool = mp.Pool(processes=nproc)
    scores = pool.map_async(_decoding,inputs).get()
    pool.close()
    pool.join()
    
#     subj_acc = np.zeros((len(subjects),))
#     scount = 0
#     i = 0
#     for subj in subjects:
#         subjmean = []
#         for cv in range(ncvs):
#             subjmean.append(scores[i])
#             i += 1
        
#         subj_acc[scount] = np.mean(subjmean)
        
#         scount += 1

#     return subj_acc
    acc = []
    for score in scores:
        acc.extend(score)
    return acc

def _decoding((trainset,testset,trainlabels,testlabels)):

# #     clf = sklearn.linear_model.LogisticRegression()
#     clf = svm.SVC(C=1.0, kernel='linear')

#     clf.fit(trainset,trainlabels)
#     predictions = clf.predict(testset)
#     acc = predictions==testlabels
#     score = np.mean(acc)

    unique_cond = np.unique(trainlabels)
    rdm = np.zeros((len(unique_cond),len(unique_cond)))
    acc = []
    for cond1 in unique_cond:
        mismatches = []
        prototype_ind = np.where(trainlabels==cond1)[0]
        prototype = np.mean(trainset[prototype_ind,:],axis=0)
        for cond2 in unique_cond:
            test_ind = np.where(testlabels==cond2)[0]
            test = np.mean(testset[test_ind,:],axis=0)
            if cond1 == cond2: 
                correct = stats.spearmanr(prototype,test)[0]
            else:
                mismatches.append(stats.spearmanr(prototype,test)[0])
        
        if correct > np.max(mismatches): 
            acc.append(1.0)
        else:
            acc.append(0.0)
    
    return acc

# 1.0 Run across subject decoding on Color Stimuli decoding
### 4-way coding for every 4 input stimuli combinations
* red, red
* red, blue
* blue, red
* blue, blue

## 1.1 Load data for "Color" stimuli

In [7]:
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'colorStim'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,inputtype)
    scount += 1

# Decode different stimulus pairings

In [12]:
nproc = 30
ncvs = 1

distances_baseline = inputStimuliDecoding(data_task, inputtype, atlasROIs=activeROIs, ncvs=ncvs, nproc=nproc)


Computing SVM classification for input stimuli: colorStim for ROI 0 / 60
Computing SVM classification for input stimuli: colorStim for ROI 1 / 60
Computing SVM classification for input stimuli: colorStim for ROI 2 / 60
Computing SVM classification for input stimuli: colorStim for ROI 3 / 60
Computing SVM classification for input stimuli: colorStim for ROI 4 / 60
Computing SVM classification for input stimuli: colorStim for ROI 5 / 60
Computing SVM classification for input stimuli: colorStim for ROI 6 / 60
Computing SVM classification for input stimuli: colorStim for ROI 7 / 60
Computing SVM classification for input stimuli: colorStim for ROI 8 / 60
Computing SVM classification for input stimuli: colorStim for ROI 9 / 60


/usr/local/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


Computing SVM classification for input stimuli: colorStim for ROI 10 / 60
Computing SVM classification for input stimuli: colorStim for ROI 11 / 60
Computing SVM classification for input stimuli: colorStim for ROI 12 / 60
Computing SVM classification for input stimuli: colorStim for ROI 13 / 60
Computing SVM classification for input stimuli: colorStim for ROI 14 / 60
Computing SVM classification for input stimuli: colorStim for ROI 15 / 60
Computing SVM classification for input stimuli: colorStim for ROI 16 / 60
Computing SVM classification for input stimuli: colorStim for ROI 17 / 60
Computing SVM classification for input stimuli: colorStim for ROI 18 / 60
Computing SVM classification for input stimuli: colorStim for ROI 19 / 60
Computing SVM classification for input stimuli: colorStim for ROI 20 / 60
Computing SVM classification for input stimuli: colorStim for ROI 21 / 60
Computing SVM classification for input stimuli: colorStim for ROI 22 / 60
Computing SVM classification for input

## 1.2 Compute statistics

In [14]:
if inputtype in ['colorStim','oriStim']:
    # Visual networks
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)


nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]
    


    

In [15]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for colorStim Stimuli: 4
Accuracies: [ 0.33333333  0.32291667  0.3125      0.31770833]


## 1.3 Map accuracies back to cortical surface

In [16]:
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    inputStim[vertex_ind,0] = statistics[roicount,0]
    inputStim[vertex_ind,1] = statistics[roicount,1]
    inputStim[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_' + inputtype + '/'
filename = inputtype + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

# 2.0 Run across subject decoding on Orientation Stimuli decoding
### 4-way coding for every 4 input stimuli combinations
* vertical, vertical
* vertical, horizontal
* horizontal, vertical
* horizontal, horizontal

## 2.1 Load data for "Ori" stimuli

In [17]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'oriStim'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,inputtype)
    scount += 1

In [18]:
nproc = 30
ncvs = 1

distances_baseline = inputStimuliDecoding(data_task, inputtype, atlasROIs=activeROIs, ncvs=ncvs, nproc=nproc)


Computing SVM classification for input stimuli: oriStim for ROI 0 / 60
Computing SVM classification for input stimuli: oriStim for ROI 1 / 60
Computing SVM classification for input stimuli: oriStim for ROI 2 / 60
Computing SVM classification for input stimuli: oriStim for ROI 3 / 60
Computing SVM classification for input stimuli: oriStim for ROI 4 / 60
Computing SVM classification for input stimuli: oriStim for ROI 5 / 60
Computing SVM classification for input stimuli: oriStim for ROI 6 / 60
Computing SVM classification for input stimuli: oriStim for ROI 7 / 60
Computing SVM classification for input stimuli: oriStim for ROI 8 / 60
Computing SVM classification for input stimuli: oriStim for ROI 9 / 60
Computing SVM classification for input stimuli: oriStim for ROI 10 / 60
Computing SVM classification for input stimuli: oriStim for ROI 11 / 60
Computing SVM classification for input stimuli: oriStim for ROI 12 / 60
Computing SVM classification for input stimuli: oriStim for ROI 13 / 60
Co

## 2.2 Compute statistics

In [19]:
if inputtype in ['colorStim','oriStim']:
    # Visual networks
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)


nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]
    


    

In [20]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for oriStim Stimuli: 22
Accuracies: [ 0.453125    0.47395833  0.36979167  0.41927083  0.30208333  0.31510417
  0.3125      0.30989583  0.359375    0.32291667  0.45572917  0.30208333
  0.45572917  0.51822917  0.36458333  0.3203125   0.30208333  0.36197917
  0.35416667  0.328125    0.29947917  0.31510417]


## 2.3 Map accuracies back to cortical surface

In [21]:
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    inputStim[vertex_ind,0] = statistics[roicount,0]
    inputStim[vertex_ind,1] = statistics[roicount,1]
    inputStim[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_' + inputtype + '/'
filename = inputtype + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

## 3.1 Load data for "Constant" stimuli

In [22]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'constantStim'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,inputtype)
    scount += 1

In [23]:
nproc = 30
ncvs = 1

distances_baseline = inputStimuliDecoding(data_task, inputtype, atlasROIs=activeROIs, ncvs=ncvs, nproc=nproc)


Computing SVM classification for input stimuli: constantStim for ROI 0 / 15
Computing SVM classification for input stimuli: constantStim for ROI 1 / 15
Computing SVM classification for input stimuli: constantStim for ROI 2 / 15
Computing SVM classification for input stimuli: constantStim for ROI 3 / 15
Computing SVM classification for input stimuli: constantStim for ROI 4 / 15
Computing SVM classification for input stimuli: constantStim for ROI 5 / 15
Computing SVM classification for input stimuli: constantStim for ROI 6 / 15
Computing SVM classification for input stimuli: constantStim for ROI 7 / 15
Computing SVM classification for input stimuli: constantStim for ROI 8 / 15
Computing SVM classification for input stimuli: constantStim for ROI 9 / 15
Computing SVM classification for input stimuli: constantStim for ROI 10 / 15
Computing SVM classification for input stimuli: constantStim for ROI 11 / 15
Computing SVM classification for input stimuli: constantStim for ROI 12 / 15
Computing

## 3.2 Compute statistics

In [24]:
if inputtype in ['colorStim','oriStim']:
    # Visual networks
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)


nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]
    


    

In [25]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for constantStim Stimuli: 14
Accuracies: [ 0.38020833  0.38541667  0.34114583  0.29427083  0.390625    0.36458333
  0.3984375   0.35677083  0.3046875   0.32291667  0.36979167  0.34635417
  0.31510417  0.32552083]


## 3.3 Map accuracies back to cortical surface

In [26]:
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    inputStim[vertex_ind,0] = statistics[roicount,0]
    inputStim[vertex_ind,1] = statistics[roicount,1]
    inputStim[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_' + inputtype + '/'
filename = inputtype + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

## 4.1 Load data for "Pitch" stimuli

In [27]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
inputtype = 'pitchStim'

scount = 0
for subj in subjNums:
#     print subj
    data_task[:,:,scount] = loadInputActivity(subj,inputtype)
    scount += 1

In [28]:
nproc = 30
ncvs = 1

distances_baseline = inputStimuliDecoding(data_task, inputtype, atlasROIs=activeROIs, ncvs=ncvs, nproc=nproc)


Computing SVM classification for input stimuli: pitchStim for ROI 0 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 1 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 2 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 3 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 4 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 5 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 6 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 7 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 8 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 9 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 10 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 11 / 15
Computing SVM classification for input stimuli: pitchStim for ROI 12 / 15
Computing SVM classification for input stimuli: 

## 4.2 Compute statistics

In [29]:
if inputtype in ['colorStim','oriStim']:
    # Visual networks
    rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)
elif inputtype in ['constantStim','pitchStim']:
    rois = np.where(networkdef==networkmappings['aud'])[0]
    rois = rois + 1
    rois = np.intersect1d(rois,activeROIs)


nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]


In [30]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', inputtype, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for pitchStim Stimuli: 15
Accuracies: [ 0.484375    0.31770833  0.31510417  0.31510417  0.49479167  0.48697917
  0.47916667  0.4453125   0.44270833  0.33854167  0.34635417  0.453125
  0.42708333  0.48697917  0.4453125 ]


## 4.3 Map accuracies back to cortical surface

In [31]:
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    inputStim[vertex_ind,0] = statistics[roicount,0]
    inputStim[vertex_ind,1] = statistics[roicount,1]
    inputStim[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_' + inputtype + '/'
filename = inputtype + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0